In [ ]:
# Packages and paths
# %%
import os
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import geopandas as gp
import scipy.stats as sp
import sys
import zipfile
import fiona

# Add the path to the Utils folder
utils_path = os.path.abspath(os.path.join('..', 'Utils'))
if utils_path not in sys.path:
    sys.path.append(utils_path)

# Now you can import the functions from CustomFunctions.py
import CustomFunctions as cf

# Local paths
datapath = '../../Data'
inputpath = '../../Data/Input'
outputpath = '../../Data/Output/Local'
shapepath = inputpath+'/Shapefiles'
figurepath = '../../Figures/Local/'

In [ ]:
folder_path = inputpath+"/NHD/"

# Don't run this if files are already unzipped

# for file in os.listdir(folder_path):
#     if file.endswith(".zip"):
#         with zipfile.ZipFile(os.path.join(folder_path, file), 'r') as zip_ref:
#             zip_ref.extractall(folder_path)

# print("Files unzipped.")

In [ ]:
# folder_path = "path_to_your_unzipped_files"
flowline_gdfs = []
point_gdfs = []
value_added_tables = []

for file in os.listdir(folder_path):
    if file.endswith(".gpkg"):
        filepath = os.path.join(folder_path, file)

        # Load specific layers
        flowlines = gp.read_file(filepath, layer="NHDFlowline")
        
        # If the value-added table is non-spatial, use pandas instead
        value_added = gp.read_file(filepath, layer="NHDPlusFlowlineVAA")  # Change to `pd.read_csv()` if needed

        flowline_gdfs.append(flowlines)
        value_added_tables.append(value_added)

# Merge the layers separately
merged_flowlines = gp.GeoDataFrame(pd.concat(flowline_gdfs, ignore_index=True))
merged_value_added = pd.concat(value_added_tables, ignore_index=True)  # Non-spatial data

# # Save to new files
# merged_flowlines.to_file("merged_flowlines.gpkg", driver="GPKG")
# merged_value_added.to_csv("merged_value_added_table.csv")  # Save as a CSV if non-spatial


In [ ]:
merged_flowlines.plot()

In [ ]:
flowlines_VAA = pd.merge(merged_flowlines,merged_value_added,
                         suffixes=['_flowlines','_VAA'], how="inner",
                         on=['nhdplusid','reachcode'])
flowlines_VAA.head()

In [ ]:
flowlines_VAA.info()

In [ ]:
# I would like to now filter to only include a certain stream order
stream_order = 2
filtered_flowlines = flowlines_VAA[flowlines_VAA["streamorde"] > stream_order]

filtered_flowlines.info()

In [ ]:
filtered_flowlines.plot()

In [ ]:
filepath = shapepath+'/NHD_H_Arizona_State_Shape/Shape/WBDHU4.shp'
hucs = gp.read_file(filepath)
hucs.head()

In [ ]:
# Check the crs
print(filtered_flowlines.crs)
print(hucs.crs)

In [ ]:
# Join the databases
flowlines_hucs = gp.sjoin(filtered_flowlines, hucs, how="inner")
flowlines_hucs.info()

# Takes about 7 minutes and if it worked, the same number as non-null values before except now it has a huc4 column

In [ ]:
flowlines_hucs.head()

In [ ]:
filepath = shapepath+'/NHD_H_Arizona_State_Shape/Shape/NHDPointEventFC.shp'
NHD_Point = gp.read_file(filepath)
NHD_Point.info()


In [ ]:
flowlines_points = pd.merge(flowlines_hucs,NHD_Point,
                         suffixes=['_flowlines','_point'], how="left",
                         on=['reachcode'])
flowlines_points.head()

In [ ]:
flowlines_points.info()

In [ ]:
flowlines_points.columns


In [ ]:
smaller_fldb = flowlines_points[['permanent_identifier', 'gnis_name','reachcode','streamorde','source_fea','huc4','name','geometry_flowlines']]
smaller_fldb

In [ ]:
smaller_fldb = smaller_fldb.rename(columns = {'geometry_flowlines':'geometry',
                                              'name':'huc_name',
                                              'source_fea':'ID'})

In [ ]:
smaller_fldb.info()

In [ ]:
smaller_fldb.plot()

In [ ]:
# Define buffer distances in meters (1 km = 1000 m)
# buffer_distances = [1000, 2000, 3000, 4000]

# # Loop through distances, apply buffer, and save each version
# for dist in buffer_distances:
#     buffered = flowlines_order4plus.copy()  # Create a copy to avoid modifying the original
#     buffered["geometry"] = flowlines_order4plus.geometry.buffer(dist)  # Apply buffer
#     buffered.to_file(f"flowlines_buffer_{dist//1000}km.shp")  # Save as shapefile

# print("Buffering and export complete!")